# Efficient Yelp API Calls (Core)
- Alexis Deviney
- 4/23/22

For this assignment, you will be working with the Yelp API.

As before, you will use the Yelp API to search your your favorite city for a cuisine type of your choice.

Extract all of the results from your search and compile them into one dataframe using a for loop as shown in the lesson "Code for Efficient API Extraction"

Save your notebook, commit the change to your repository and submit the repository URL for this assignment.



In [1]:
import numpy as np
import pandas as pd
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# Load API Credentials
with open('/Users/Alexis/.secret/yelp_api.json') as f:
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [3]:
LOCATION = 'San Francisco, CA 94104'
TERM = 'Vegan'

In [5]:
JSON_FILE = f"Data/results_in_progress_Vegan_food_SF.json"
JSON_FILE

'Data/results_in_progress_Vegan_food_SF.json'

In [6]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    
    ## save the first page of results
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
## If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_Vegan_food_SF.json not found. Saving empty list to file.


In [7]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [8]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [9]:
## How many results total?
total_results = results['total']
total_results

628

In [10]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [11]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((total_results-n_results)/ results_per_page)
n_pages

32

In [12]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [13]:
for i in tqdm_notebook( range(1,n_pages+1)):
    time.sleep(.2)
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
#     display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)

  0%|          | 0/32 [00:00<?, ?it/s]

In [14]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,M1BK_5O_jWRIdlnnQy0yfA,vida-cantina-sf-san-francisco,Vida Cantina SF,https://s3-media2.fl.yelpcdn.com/bphoto/iks36D...,False,https://www.yelp.com/biz/vida-cantina-sf-san-f...,100,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.5,"{'latitude': 37.79147, 'longitude': -122.40366}","[delivery, restaurant_reservation, pickup]",$$,"{'address1': '56 Belden Ln', 'address2': None,...",+14155253042,(415) 525-3042,170.001073
1,9DOYWliUAN9S-ECTZp20XQ,hopstone-farm-and-craft-san-francisco,Hopstone - Farm & Craft,https://s3-media3.fl.yelpcdn.com/bphoto/5-iuwj...,False,https://www.yelp.com/biz/hopstone-farm-and-cra...,7,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.5,"{'latitude': 37.7936, 'longitude': -122.39991}","[delivery, pickup]",NaN,"{'address1': '222 Battery St', 'address2': Non...",+14155362822,(415) 536-2822,359.372300
2,5Mqsd52IEJtV9dUJAhcTlQ,lucky-creation-vegetarian-restaurant-san-franc...,Lucky Creation Vegetarian Restaurant,https://s3-media1.fl.yelpcdn.com/bphoto/UZY3Qd...,False,https://www.yelp.com/biz/lucky-creation-vegeta...,566,"[{'alias': 'vegetarian', 'title': 'Vegetarian'...",4.0,"{'latitude': 37.7951059659376, 'longitude': -1...","[delivery, pickup]",$,"{'address1': '854 Washington St', 'address2': ...",+14159890818,(415) 989-0818,671.455340
3,1EWkmc_myDLYfvorOnYzEA,hinodeya-downtown-san-francisco-3,Hinodeya Downtown,https://s3-media3.fl.yelpcdn.com/bphoto/l3ZNeY...,False,https://www.yelp.com/biz/hinodeya-downtown-san...,322,"[{'alias': 'ramen', 'title': 'Ramen'}, {'alias...",4.5,"{'latitude': 37.794597, 'longitude': -122.404432}","[delivery, pickup]",$$,"{'address1': '680 Clay St', 'address2': '', 'a...",+14152006466,(415) 200-6466,467.799356
4,XKj8_4QdoBMeYcg6OoespA,enjoy-vegetarian-restaurant-san-francisco-2,Enjoy Vegetarian Restaurant,https://s3-media2.fl.yelpcdn.com/bphoto/2NtAI4...,False,https://www.yelp.com/biz/enjoy-vegetarian-rest...,571,"[{'alias': 'chinese', 'title': 'Chinese'}]",4.0,"{'latitude': 37.7959058, 'longitude': -122.405...","[delivery, pickup]",$$,"{'address1': '839 Kearny St', 'address2': '', ...",+14159567868,(415) 956-7868,625.903147


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
623,J3N0MdRMW2cJdq8PHXH0ig,rambler-san-francisco-2,Rambler,https://s3-media1.fl.yelpcdn.com/bphoto/vYKeJj...,False,https://www.yelp.com/biz/rambler-san-francisco...,456,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",3.5,"{'latitude': 37.78771, 'longitude': -122.41071}",[delivery],$$,"{'address1': '545 Post St', 'address2': '', 'a...",+14155498008,(415) 549-8008,850.907364
624,dPKf5l7vCDZIrWQPEHVTEg,peets-coffee-san-francisco-38,Peet's Coffee,https://s3-media2.fl.yelpcdn.com/bphoto/-79LYj...,False,https://www.yelp.com/biz/peets-coffee-san-fran...,33,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",4.0,"{'latitude': 37.791925, 'longitude': -122.3928...",[delivery],$$,"{'address1': '121 Spear St', 'address2': '', '...",+16282203250,(628) 220-3250,804.883495
625,6dbvfyQnSmPACipuqgEZdw,little-paradise-candy-store-san-francisco,Little Paradise Candy Store,https://s3-media3.fl.yelpcdn.com/bphoto/LXart7...,False,https://www.yelp.com/biz/little-paradise-candy...,3,"[{'alias': 'gourmet', 'title': 'Specialty Food'}]",3.5,"{'latitude': 37.7959022521973, 'longitude': -1...",[],NaN,"{'address1': '1101 Stockton St', 'address2': '...",+14159860680,(415) 986-0680,805.670342
626,Qb0rINwKnoMOPp9RpebU-Q,annies-hot-dog-cart-san-francisco-2,Annie's Hot Dog Cart,https://s3-media2.fl.yelpcdn.com/bphoto/WrnqfA...,False,https://www.yelp.com/biz/annies-hot-dog-cart-s...,6,"[{'alias': 'foodstands', 'title': 'Food Stands'}]",4.0,"{'latitude': 37.785692807166, 'longitude': -12...",[],$,"{'address1': 'Ellis & Market', 'address2': '',...",+14158610778,(415) 861-0778,1152.402444
627,KTmeAHhB3u51NETxuk7iQg,starbucks-san-francisco-234,Starbucks,https://s3-media2.fl.yelpcdn.com/bphoto/hJFUWE...,False,https://www.yelp.com/biz/starbucks-san-francis...,67,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",2.0,"{'latitude': 37.7844609656521, 'longitude': -1...",[delivery],$,"{'address1': '789 Mission St', 'address2': '',...",+14153436272,(415) 343-6272,735.781776


In [15]:
# check for duplicate IDs
final_df.duplicated(subset='id').sum()

4

In [22]:
final_df = final_df.drop_duplicates(subset='id')

In [23]:
# save the final results
final_df.to_csv('Data/final_results_vegan_food_SF.csv',index=False)